# 使用TensorBoard可视化Fashion-MNIST数据集

本notebook展示如何使用TensorBoard替代matplotlib来可视化Fashion-MNIST数据集。

## 安装依赖

首先确保安装了必要的包：
```bash
pip install torch torchvision tensorboard
```

In [1]:
# 导入必要的库
import torch
import torchvision
from torch.utils import data
from torchvision import transforms
from torch.utils.tensorboard import SummaryWriter
import numpy as np
from datetime import datetime
import os

# 设置随机种子
torch.manual_seed(42)
print('✅ 库导入完成')

✅ 库导入完成


In [2]:
def get_fashion_mnist_labels(labels):
    """返回Fashion-MNIST数据集的文本标签"""
    text_labels = ['t-shirt', 'trouser', 'pullover', 'dress', 'coat',
                   'sandal', 'shirt', 'sneaker', 'bag', 'ankle boot']
    return [text_labels[int(i)] for i in labels]

print('✅ 标签函数定义完成')

✅ 标签函数定义完成


In [3]:
# 加载Fashion-MNIST数据集
print('正在加载Fashion-MNIST数据集...')

# 数据变换
trans = transforms.ToTensor()

# 加载数据集
mnist_train = torchvision.datasets.FashionMNIST(
    root="./Machine-Learning/data", train=True, transform=trans, download=True)

mnist_test = torchvision.datasets.FashionMNIST(
    root="./Machine-Learning/data", train=False, transform=trans, download=True)

print(f'训练集大小: {len(mnist_train)}')
print(f'测试集大小: {len(mnist_test)}')
print(f'图像形状: {mnist_train[0][0].shape}')
print('✅ 数据集加载完成')

正在加载Fashion-MNIST数据集...
训练集大小: 60000
测试集大小: 10000
图像形状: torch.Size([1, 28, 28])
✅ 数据集加载完成


## 1. 基础TensorBoard可视化

创建TensorBoard writer并添加单个图像样本：

In [4]:
# 创建TensorBoard日志目录
log_dir = f"runs/fashion_mnist_{datetime.now().strftime('%Y%m%d_%H%M%S')}"
writer = SummaryWriter(log_dir)

print(f'TensorBoard日志保存在: {log_dir}')
print('启动TensorBoard命令: tensorboard --logdir=runs')

# 添加单个样本
print('添加单个样本到TensorBoard...')
for i in range(10):
    img, label = mnist_train[i]
    label_text = get_fashion_mnist_labels([label])[0]
    
    # 添加图像到TensorBoard
    writer.add_image(f'Single_Samples/{i:02d}_{label_text}', img, 0)

print('✅ 单个样本添加完成')

TensorBoard日志保存在: runs/fashion_mnist_20250914_171846
启动TensorBoard命令: tensorboard --logdir=runs
添加单个样本到TensorBoard...
✅ 单个样本添加完成


## 2. 创建图像网格

这相当于原始代码中的 `show_images` 函数功能：

In [5]:
# 获取一批数据 (相当于原始代码中的批量获取)
batch_size = 18
train_loader = data.DataLoader(mnist_train, batch_size=batch_size, shuffle=True)
X, y = next(iter(train_loader))

print(f'批量数据形状: {X.shape}')
print(f'标签: {y}')

# 获取标签文本
labels_text = get_fashion_mnist_labels(y)
print(f'标签文本: {labels_text}')

# 创建图像网格 (类似原始代码的 show_images 功能)
img_grid = torchvision.utils.make_grid(
    X.reshape(batch_size, 1, 28, 28),  # 重塑为正确形状
    nrow=6,  # 每行6张图片 (类似原始代码的 2行9列)
    normalize=True,
    scale_each=True,
    pad_value=1
)

# 添加到TensorBoard
writer.add_image('Fashion_MNIST_Grid', img_grid, 0)

print('✅ 图像网格创建完成')

批量数据形状: torch.Size([18, 1, 28, 28])
标签: tensor([5, 7, 4, 7, 3, 8, 9, 5, 3, 1, 2, 3, 2, 3, 3, 7, 9, 9])
标签文本: ['sandal', 'sneaker', 'coat', 'sneaker', 'dress', 'bag', 'ankle boot', 'sandal', 'dress', 'trouser', 'pullover', 'dress', 'pullover', 'dress', 'dress', 'sneaker', 'ankle boot', 'ankle boot']
✅ 图像网格创建完成


## 3. 按类别分组可视化

为每个Fashion-MNIST类别创建单独的可视化：

In [6]:
# 按类别收集样本
print('按类别收集样本...')
class_samples = {}

for i, (img, label) in enumerate(mnist_train):
    label_int = int(label)
    if label_int not in class_samples:
        class_samples[label_int] = []
    if len(class_samples[label_int]) < 6:  # 每个类别6个样本
        class_samples[label_int].append(img)
    
    # 所有类别都收集够了就停止
    if all(len(samples) >= 6 for samples in class_samples.values()) and len(class_samples) == 10:
        break

# 为每个类别创建网格
for label_int, images in class_samples.items():
    if len(images) > 0:
        label_text = get_fashion_mnist_labels([label_int])[0]
        class_grid = torchvision.utils.make_grid(
            torch.stack(images), 
            nrow=3,
            normalize=True,
            scale_each=True,
            pad_value=1
        )
        writer.add_image(f'By_Class/{label_int:02d}_{label_text}', class_grid, 0)

print('✅ 类别分组可视化完成')

按类别收集样本...
✅ 类别分组可视化完成


## 4. 添加数据集统计信息

TensorBoard的优势：可以添加各种统计图表

In [7]:
# 计算类别分布
print('计算数据集统计信息...')
class_counts = torch.zeros(10)
for _, label in mnist_train:
    class_counts[label] += 1

# 添加直方图
writer.add_histogram('Dataset/Class_Distribution', class_counts, 0)

# 添加标量统计
writer.add_scalar('Dataset/Total_Samples', len(mnist_train), 0)
writer.add_scalar('Dataset/Num_Classes', 10, 0)
writer.add_scalar('Dataset/Image_Height', 28, 0)
writer.add_scalar('Dataset/Image_Width', 28, 0)

# 像素值分布
sample_batch = torch.stack([mnist_train[i][0] for i in range(1000)])
writer.add_histogram('Dataset/Pixel_Values', sample_batch, 0)

# 添加文本信息
class_names = "\n".join([f"{i}: {name}" for i, name in enumerate(
    ['t-shirt', 'trouser', 'pullover', 'dress', 'coat',
     'sandal', 'shirt', 'sneaker', 'bag', 'ankle boot']
)])
writer.add_text('Dataset/Class_Names', class_names, 0)

print('✅ 统计信息添加完成')

计算数据集统计信息...
✅ 统计信息添加完成


## 5. 完成并启动TensorBoard

In [9]:
# 关闭writer
writer.close()

print('\n✅ TensorBoard可视化完成!')
print(f'📁 日志目录: {log_dir}')
print('🚀 启动TensorBoard: tensorboard --logdir=runs')
print('🌐 然后在浏览器中打开: http://localhost:6006')

print('\n=== 使用说明 ===')
print('1. 在终端运行: tensorboard --logdir=runs')
print('2. 在浏览器中打开: http://localhost:6006')
print('3. 在TensorBoard界面中切换不同的标签页:')
print('   - IMAGES: 查看图像可视化')
print('   - HISTOGRAMS: 查看数据分布')
print('   - SCALARS: 查看数值统计')
print('   - TEXT: 查看文本信息')


✅ TensorBoard可视化完成!
📁 日志目录: runs/fashion_mnist_20250914_171846
🚀 启动TensorBoard: tensorboard --logdir=runs
🌐 然后在浏览器中打开: http://localhost:6006

=== 使用说明 ===
1. 在终端运行: tensorboard --logdir=runs
2. 在浏览器中打开: http://localhost:6006
3. 在TensorBoard界面中切换不同的标签页:
   - IMAGES: 查看图像可视化
   - HISTOGRAMS: 查看数据分布
   - SCALARS: 查看数值统计
   - TEXT: 查看文本信息


## 6. 在Jupyter中直接启动TensorBoard (可选)

如果你想在Jupyter notebook中直接查看TensorBoard，可以使用以下命令：

In [10]:
# 在Jupyter中启动TensorBoard (需要安装 jupyter-tensorboard)
# 取消注释下面的代码来使用:

%load_ext tensorboard
%tensorboard --logdir runs

print('如果要在Jupyter中直接显示TensorBoard:')
print('1. 安装: pip install jupyter-tensorboard')
print('2. 取消注释上面的代码并运行')

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6006 (pid 16224), started 0:01:12 ago. (Use '!kill 16224' to kill it.)

如果要在Jupyter中直接显示TensorBoard:
1. 安装: pip install jupyter-tensorboard
2. 取消注释上面的代码并运行


## 7. 对比原始方法

### 原始matplotlib方法:
```python
X, y = next(iter(data.DataLoader(mnist_train, batch_size=18)))
show_images(X.reshape(18, 28, 28), 2, 9, titles=get_fashion_mnist_labels(y))
```

### TensorBoard方法的优势:

1. **交互性**: 可以缩放、平移图像
2. **组织性**: 可以按类别、时间等维度组织
3. **统计性**: 可以添加直方图、标量等统计信息
4. **持久性**: 结果保存在磁盘上，可以随时查看
5. **专业性**: 专为深度学习设计的可视化工具
6. **扩展性**: 支持模型图、嵌入等高级可视化

### 使用场景:

- **快速查看**: 使用matplotlib
- **深入分析**: 使用TensorBoard
- **模型训练**: TensorBoard可以实时监控训练过程